<a href="https://colab.research.google.com/github/Ananya-AJ/CMPE255-Data-Mining/blob/main/Assignment4/k_data_augmentation_classification_tabulardata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We'll use the UCI Credit Card Dataset. In this dataset, we are predicting whether or not a credit card client will default on their payment. The target variable is a binary variable, where 1 indicates that the client defaulted and 0 indicates that the client did not default

In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls

--2023-04-09 23:17:36--  https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5539328 (5.3M) [application/x-httpd-php]
Saving to: ‘default of credit card clients.xls’

default of credit c 100%[===================>]   5.28M  9.34MB/s    in 0.6s    

2023-04-09 23:17:37 (9.34 MB/s) - ‘default of credit card clients.xls’ saved [5539328/5539328]



In [2]:
import pandas as pd
df = pd.read_excel('default of credit card clients.xls', header=1)

preprocess the data by splitting it into a training and testing set, normalizing the features, and one-hot encoding the target variable

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# Separate the features and target variable
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Split the data into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# One-hot encode the target variable
encoder = OneHotEncoder()
y_train = encoder.fit_transform(y_train.reshape(-1, 1)).toarray()
y_test = encoder.transform(y_test.reshape(-1, 1)).toarray()

Before Augmentation

In [5]:
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(y_train.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
750/750 [==============================] - 5s 5ms/step - loss: 0.4693 - accuracy: 0.8068 - val_loss: 0.4474 - val_accuracy: 0.8147
Epoch 2/10
750/750 [==============================] - 5s 7ms/step - loss: 0.4415 - accuracy: 0.8181 - val_loss: 0.4452 - val_accuracy: 0.8170
Epoch 3/10
750/750 [==============================] - 3s 3ms/step - loss: 0.4353 - accuracy: 0.8200 - val_loss: 0.4380 - val_accuracy: 0.8205
Epoch 4/10
750/750 [==============================] - 2s 2ms/step - loss: 0.4317 - accuracy: 0.8207 - val_loss: 0.4390 - val_accuracy: 0.8177
Epoch 5/10
750/750 [==============================] - 2s 2ms/step - loss: 0.4282 - accuracy: 0.8213 - val_loss: 0.4382 - val_accuracy: 0.8195
Epoch 6/10
750/750 [==============================] - 2s 2ms/step - loss: 0.4270 - accuracy: 0.8207 - val_loss: 0.4388 - val_accuracy: 0.8175
Epoch 7/10
750/750 [==============================] - 2s 3ms/step - loss: 0.4237 - accuracy: 0.8234 - val_loss: 0.4372 - val_accuracy: 0.8170
Epoch 

In [6]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy)

188/188 [==============================] - 0s 2ms/step - loss: 0.4356 - accuracy: 0.8150
Test accuracy: 0.8149999976158142


Data augmentation: let's use noise injection as a data augmentation technique on the credit card dataset.

One way to add noise to the data is by adding random Gaussian noise to each feature with a given mean and sd

In [3]:
import numpy as np

def add_noise(feature, mean, std):
    noise = np.random.normal(mean, std, len(feature))
    return feature + noise

# Add noise to each feature of the dataset
for i in range(1, len(df.columns)):
    df.iloc[:, i] = add_noise(df.iloc[:, i], 0, 0.1)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
# Separate features and labels
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# Split the data into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
# Define the neural network model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=2)

# Evaluate the model on the testing set
score = model.evaluate(X_test, y_test, verbose=0)
print("Test accuracy:", score[1])

Added random Gaussian noise with mean 0 and standard deviation 0.1 to each feature of the dataset, except for the target variable.

we infer that adding noise to the data can help in making the model more robust to noise in the input data, but it may not always improve the performance of the model. It's always a good idea to try different data augmentation techniques and see which ones work best for your particular problem.

It reduces loss but worsens test accuracy